In [646]:
import warnings
warnings.filterwarnings('ignore')

# 회원 탈퇴 예측

### 1 : 데이터 읽고 이용데이터 수정

In [647]:
# use_log_months.csv & customer_join.csv


import pandas as pd

customer = pd.read_csv("./data/customer_join.csv")
use_months = pd.read_csv("./data/use_log_months.csv")


In [648]:
customer.head()

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,OA832399,XXXX,C01,F,2015-05-01,NaN,CA1,0,종일,10500,일반,4.833333,5.0,8,2,1,2019-04-30,47
1,PL270116,XXXXX,C01,M,2015-05-01,NaN,CA1,0,종일,10500,일반,5.083333,5.0,7,3,1,2019-04-30,47
2,OA974876,XXXXX,C01,M,2015-05-01,NaN,CA1,0,종일,10500,일반,4.583333,5.0,6,3,1,2019-04-30,47
3,HD024127,XXXXX,C01,F,2015-05-01,NaN,CA1,0,종일,10500,일반,4.833333,4.5,7,2,1,2019-04-30,47
4,HD661448,XXXXX,C03,F,2015-05-01,NaN,CA1,0,야간,6000,일반,3.916667,4.0,6,1,1,2019-04-30,47


In [649]:
use_months

,연월,customer_id,count
0,201804,AS002855,4
1,201804,AS009013,2
2,201804,AS009373,3
3,201804,AS015315,6
4,201804,AS015739,7
...,...,...,...
36837,201903,TS995853,8
36838,201903,TS998593,8
36839,201903,TS999079,3
36840,201903,TS999231,6


In [650]:
customer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4192 entries, 0 to 4191
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   customer_id        4192 non-null   object 
 1   name               4192 non-null   object 
 2   class              4192 non-null   object 
 3   gender             4192 non-null   object 
 4   start_date         4192 non-null   object 
 5   end_date           1350 non-null   object 
 6   campaign_id        4192 non-null   object 
 7   is_deleted         4192 non-null   int64  
 8   class_name         4192 non-null   object 
 9   price              4192 non-null   int64  
 10  campaign_name      4192 non-null   object 
 11  mean               4192 non-null   float64
 12  median             4192 non-null   float64
 13  max                4192 non-null   int64  
 14  min                4192 non-null   int64  
 15  routine_flg        4192 non-null   int64  
 16  calc_date          4192 

In [651]:
use_months.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36842 entries, 0 to 36841
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   연월           36842 non-null  int64 
 1   customer_id  36842 non-null  object
 2   count        36842 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 863.6+ KB


In [652]:
# 이번달과 1개월 전의 이용횟수


year_months = list(use_months["연월"].unique())
uselog = pd.DataFrame()

for i in range(1, len(year_months)):
    # 5월
    tmp = use_months.loc[use_months['연월'] == year_months[i]]
    tmp.rename(columns={"count": "count_0"}, inplace=True)
    
    
    # 4월
    tmp_before = use_months.loc[use_months['연월'] == year_months[i-1]]
    tmp_before.rename(columns={"count": "count_1"}, inplace=True)

    del tmp_before['연월']

    tmp = pd.merge(tmp, tmp_before, on='customer_id', how='left')
    uselog = pd.concat([uselog, tmp], ignore_index=True)

uselog

,연월,customer_id,count_0,count_1
0,201805,AS002855,5,4.0
1,201805,AS009373,4,3.0
2,201805,AS015233,7,NaN
3,201805,AS015315,3,6.0
4,201805,AS015739,5,7.0
...,...,...,...,...
33846,201903,TS995853,8,11.0
33847,201903,TS998593,8,7.0
33848,201903,TS999079,3,2.0
33849,201903,TS999231,6,6.0


In [653]:
# uselog1 = pd.merge(uselog,customer,on='customer_id',how='left')
# uselog1

### 2 : 탈퇴전월의 탈퇴고객데이터 작성

In [654]:
from dateutil.relativedelta import relativedelta


# customer에 is_deleted가 1인것을 exit_customer에 넣음
exit_customer = customer.loc[customer["is_deleted"] == 1]

# exit_date 컬럼을 만듦
exit_customer['exit_date'] = None
# end_date의 컬럼을 datetime으로 변환
exit_customer['end_date'] = pd.to_datetime(exit_customer['end_date']) 


# exit_date[i] = end_date의 1개월 전을 계산 -> end_date[i] - 1 month
for i in range(len(exit_customer)):
    exit_customer['exit_date'].iloc[i] = exit_customer['end_date'].iloc[i] - relativedelta(months=1)
    
exit_customer   

# 연월 컬럼을 exit_date로 변경
exit_customer['연월'] = pd.to_datetime(exit_customer['exit_date']).dt.strftime("%Y%m")
uselog['연월'] = uselog['연월'].astype(str)


# uselog와 exit_customer 를 merge : 기준이 customer_id, 연월
exit_uselog = pd.merge(uselog,exit_customer,on=['customer_id','연월'],how='left')


print(len(uselog))
exit_uselog





33851


,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201805,AS002855,5,4.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,201805,AS009373,4,3.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,201805,AS015233,7,NaN,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,201805,AS015315,3,6.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,201805,AS015739,5,7.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33846,201903,TS995853,8,11.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33847,201903,TS998593,8,7.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33848,201903,TS999079,3,2.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33849,201903,TS999231,6,6.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [655]:
exit_uselog = exit_uselog.dropna(subset="name")
print(len(exit_uselog))
print(len(exit_uselog["customer_id"].unique()))
exit_uselog.head()

1104
1104


,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
19,201805,AS055680,3,3.0,XXXXX,C01,M,2018-03-01,2018-06-30,CA1,...,10500.0,일반,3.000000,3.0,3.0,3.0,0.0,2018-06-30,3.0,2018-05-30 00:00:00
57,201805,AS169823,2,3.0,XX,C01,M,2017-11-01,2018-06-30,CA1,...,10500.0,일반,3.000000,3.0,4.0,2.0,1.0,2018-06-30,7.0,2018-05-30 00:00:00
110,201805,AS305860,5,3.0,XXXX,C01,M,2017-06-01,2018-06-30,CA1,...,10500.0,일반,3.333333,3.0,5.0,2.0,0.0,2018-06-30,12.0,2018-05-30 00:00:00
128,201805,AS363699,5,3.0,XXXXX,C01,M,2018-02-01,2018-06-30,CA1,...,10500.0,일반,3.333333,3.0,5.0,2.0,0.0,2018-06-30,4.0,2018-05-30 00:00:00
147,201805,AS417696,1,4.0,XX,C03,F,2017-09-01,2018-06-30,CA1,...,6000.0,일반,2.000000,1.0,4.0,1.0,0.0,2018-06-30,9.0,2018-05-30 00:00:00


### 3 : 지속회원의 데이터 작성

In [656]:
# 지속 회원 추출 
# usglog 데이터 결합

In [657]:
conti_customer = customer.loc[customer['is_deleted']==0]
conti_uselog = pd.merge(uselog,conti_customer,on="customer_id",how="left")
print(len(conti_uselog))
conti_uselog = conti_uselog.dropna(subset="name")
print(len(conti_uselog))

33851
27422


In [658]:
# 지속 회원 데이터를 회원당 1개가 되게 언더샘플링

conti_uselog = conti_uselog.sample(frac=1).reset_index(drop=True)
conti_uselog = conti_uselog.drop_duplicates(subset="customer_id")
conti_uselog


,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,201808,GD373372,4,6.0,XXXX,C03,F,2016-07-01,NaN,CA2,...,야간,6000.0,입회비반액할인,5.000000,5.0,8.0,2.0,1.0,2019-04-30,33.0
1,201903,HI216184,3,2.0,XXXX,C01,M,2015-06-01,NaN,CA1,...,종일,10500.0,일반,4.000000,4.0,7.0,2.0,1.0,2019-04-30,46.0
2,201811,TS250786,6,5.0,XXXXXX,C01,F,2016-06-01,NaN,CA1,...,종일,10500.0,일반,5.000000,5.0,6.0,4.0,1.0,2019-04-30,34.0
3,201812,IK619199,5,4.0,XXXX,C01,M,2017-07-01,NaN,CA2,...,종일,10500.0,입회비반액할인,5.583333,6.0,7.0,2.0,1.0,2019-04-30,21.0
4,201806,HI099779,8,NaN,XXXXX,C01,M,2018-06-04,NaN,CA2,...,종일,10500.0,입회비반액할인,6.700000,7.0,9.0,4.0,1.0,2019-04-30,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24552,201903,TS566915,9,14.0,XXXXX,C03,M,2019-02-07,NaN,CA1,...,야간,6000.0,일반,11.500000,11.5,14.0,9.0,1.0,2019-04-30,2.0
25145,201903,GD625169,8,NaN,XXXX,C03,F,2019-03-06,NaN,CA1,...,야간,6000.0,일반,8.000000,8.0,8.0,8.0,0.0,2019-04-30,1.0
25587,201903,HI785400,11,NaN,XXXXX,C02,F,2019-03-05,NaN,CA1,...,주간,7500.0,일반,11.000000,11.0,11.0,11.0,1.0,2019-04-30,1.0
26020,201903,PL425572,7,NaN,XXXX,C01,F,2019-03-12,NaN,CA1,...,종일,10500.0,일반,7.000000,7.0,7.0,7.0,0.0,2019-04-30,1.0


In [659]:
# 탈퇴회원 + 지속회원

predict_data = pd.concat([conti_uselog,exit_uselog],ignore_index=True)
print(len(predict_data))
predict_data.head()

3946


,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201808,GD373372,4,6.0,XXXX,C03,F,2016-07-01,NaT,CA2,...,6000.0,입회비반액할인,5.000000,5.0,8.0,2.0,1.0,2019-04-30,33.0,NaN
1,201903,HI216184,3,2.0,XXXX,C01,M,2015-06-01,NaT,CA1,...,10500.0,일반,4.000000,4.0,7.0,2.0,1.0,2019-04-30,46.0,NaN
2,201811,TS250786,6,5.0,XXXXXX,C01,F,2016-06-01,NaT,CA1,...,10500.0,일반,5.000000,5.0,6.0,4.0,1.0,2019-04-30,34.0,NaN
3,201812,IK619199,5,4.0,XXXX,C01,M,2017-07-01,NaT,CA2,...,10500.0,입회비반액할인,5.583333,6.0,7.0,2.0,1.0,2019-04-30,21.0,NaN
4,201806,HI099779,8,NaN,XXXXX,C01,M,2018-06-04,NaT,CA2,...,10500.0,입회비반액할인,6.700000,7.0,9.0,4.0,1.0,2019-04-30,10.0,NaN


### 4 : 예측할 달의 재적기간 작성

In [660]:
# 재적기간 = 연월컬럼 - start_date

predict_data['preiod'] = 0

predict_data['now_date'] = pd.to_datetime(predict_data['연월'], format='%Y%m')
predict_data['start_date'] = pd.to_datetime(predict_data['start_date'])


# customoer_join의 행을 순차적으로 돌면서 calc_date와 start_date의 차이 값을 구함
for i in range(len(predict_data)):
    delta = relativedelta(predict_data['now_date'].iloc[i],
                          predict_data['start_date'].iloc[i])
    
    predict_data['preiod'].iloc[i] = delta.years*12 + delta.months 
    
predict_data

,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date,preiod,now_date
0,201808,GD373372,4,6.0,XXXX,C03,F,2016-07-01,NaT,CA2,...,5.000000,5.0,8.0,2.0,1.0,2019-04-30,33.0,NaN,25,2018-08-01
1,201903,HI216184,3,2.0,XXXX,C01,M,2015-06-01,NaT,CA1,...,4.000000,4.0,7.0,2.0,1.0,2019-04-30,46.0,NaN,45,2019-03-01
2,201811,TS250786,6,5.0,XXXXXX,C01,F,2016-06-01,NaT,CA1,...,5.000000,5.0,6.0,4.0,1.0,2019-04-30,34.0,NaN,29,2018-11-01
3,201812,IK619199,5,4.0,XXXX,C01,M,2017-07-01,NaT,CA2,...,5.583333,6.0,7.0,2.0,1.0,2019-04-30,21.0,NaN,17,2018-12-01
4,201806,HI099779,8,NaN,XXXXX,C01,M,2018-06-04,NaT,CA2,...,6.700000,7.0,9.0,4.0,1.0,2019-04-30,10.0,NaN,0,2018-06-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3941,201902,TS645212,4,2.0,XXXX,C03,F,2018-03-01,2019-03-31,CA1,...,4.500000,4.5,7.0,1.0,0.0,2019-03-31,12.0,2019-02-28 00:00:00,11,2019-02-01
3942,201902,TS741703,5,6.0,XXXX,C03,M,2018-12-08,2019-03-31,CA3,...,6.250000,6.0,8.0,5.0,0.0,2019-03-31,3.0,2019-02-28 00:00:00,1,2019-02-01
3943,201902,TS859258,1,3.0,XXXXX,C02,F,2018-12-07,2019-03-31,CA3,...,2.500000,2.0,5.0,1.0,0.0,2019-03-31,3.0,2019-02-28 00:00:00,1,2019-02-01
3944,201902,TS886985,5,3.0,XXX,C02,F,2018-03-01,2019-03-31,CA1,...,4.250000,4.0,7.0,2.0,1.0,2019-03-31,12.0,2019-02-28 00:00:00,11,2019-02-01


### 5 : 결측치 제거

In [661]:
# end_date, exit_date, count_1 결측치 있음 하지만 end_date,exit_date의 유지 회원은 결측치가 아님 삭제 X

predict_data.isna().sum()

연월                      0
customer_id             0
count_0                 0
count_1               254
name                    0
class                   0
gender                  0
start_date              0
end_date             2842
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
exit_date            2842
preiod                  0
now_date                0
dtype: int64

In [662]:
predict_data = predict_data.dropna(subset=['count_1'])
predict_data.isna().sum()

연월                      0
customer_id             0
count_0                 0
count_1                 0
name                    0
class                   0
gender                  0
start_date              0
end_date             2640
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
exit_date            2640
preiod                  0
now_date                0
dtype: int64

### 6 : 문자열 변수를 처리할수 있도록 가공

In [663]:
# 가입 캠페인 구분 , 회원구분, 성별 등의 카테고리 변수 처리
# X : 1개월 전의 이용횟수 (count1) , campaign_name, class_name, gender, 정기이용 여부 플래그 (routine_flg), 재적기간(period)
# Y : 탈퇴 플래그 (is_deleted)

In [664]:
target_col = ['campaign_name','class_name','gender','count_1','routine_flg','preiod','is_deleted']
predict_data = predict_data[target_col]
predict_data.head()

,campaign_name,class_name,gender,count_1,routine_flg,preiod,is_deleted
0,입회비반액할인,야간,F,6.0,1.0,25,0.0
1,일반,종일,M,2.0,1.0,45,0.0
2,일반,종일,F,5.0,1.0,29,0.0
3,입회비반액할인,종일,M,4.0,1.0,17,0.0
5,입회비반액할인,종일,F,6.0,1.0,30,0.0


In [665]:
predict_data = pd.get_dummies(predict_data)
predict_data.head()

,count_1,routine_flg,preiod,is_deleted,campaign_name_일반,campaign_name_입회비무료,campaign_name_입회비반액할인,class_name_야간,class_name_종일,class_name_주간,gender_F,gender_M
0,6.0,1.0,25,0.0,False,False,True,True,False,False,True,False
1,2.0,1.0,45,0.0,True,False,False,False,True,False,False,True
2,5.0,1.0,29,0.0,True,False,False,False,True,False,True,False
3,4.0,1.0,17,0.0,False,False,True,False,True,False,False,True
5,6.0,1.0,30,0.0,False,False,True,False,True,False,True,False


In [666]:
del predict_data['campaign_name_일반']
del predict_data['class_name_야간']
del predict_data['gender_M']

predict_data.head()

,count_1,routine_flg,preiod,is_deleted,campaign_name_입회비무료,campaign_name_입회비반액할인,class_name_종일,class_name_주간,gender_F
0,6.0,1.0,25,0.0,False,True,False,False,True
1,2.0,1.0,45,0.0,False,False,True,False,False
2,5.0,1.0,29,0.0,False,False,True,False,True
3,4.0,1.0,17,0.0,False,True,True,False,False
5,6.0,1.0,30,0.0,False,True,True,False,True


### 7 : 의사결정나무를 사용해서 탈퇴예측모델을 작성하자

In [667]:
from sklearn.tree import DecisionTreeClassifier
import sklearn.model_selection
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# 탈퇴 데이터와 유지 데이터의 개수를 정리
# 유지 데이터 : 2842개 탈퇴 데이터 : 1104개 -> 유지데이터에서 임의로 1104건을 추출해서 50:50 비율을 맞춤

exit = predict_data.loc[predict_data["is_deleted"] == 1]

conti = predict_data.loc[predict_data["is_deleted"] == 0].sample(len(exit))
# print(len(conti))
exit_conti = pd.concat([exit,conti],ignore_index=True) 
exit_conti


,count_1,routine_flg,preiod,is_deleted,campaign_name_입회비무료,campaign_name_입회비반액할인,class_name_종일,class_name_주간,gender_F
0,3.0,0.0,2,1.0,False,False,True,False,False
1,3.0,1.0,6,1.0,False,False,True,False,False
2,3.0,0.0,11,1.0,False,False,True,False,False
3,3.0,0.0,3,1.0,False,False,True,False,False
4,4.0,0.0,8,1.0,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...
2099,12.0,1.0,0,0.0,False,False,False,True,True
2100,6.0,1.0,24,0.0,False,False,False,True,True
2101,1.0,1.0,37,0.0,False,False,False,False,False
2102,6.0,1.0,9,0.0,False,True,False,False,False


In [668]:
# X : 1개월 전의 이용횟수 (count1) , campaign_name, class_name, gender, 정기이용 여부 플래그 (routine_flg), 재적기간(period)
# Y : 탈퇴 플래그 (is_deleted)

X = exit_conti[["count_1","routine_flg","preiod","campaign_name_입회비무료","campaign_name_입회비반액할인","class_name_종일","class_name_주간","gender_F"]]

y = exit_conti['is_deleted']




X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=0)

# 훈련 
df_clf = DecisionTreeClassifier()
df_clf.fit(X_train,y_train)
y_test_pred = df_clf.predict(X_test)
print(y_test_pred) # 0은 지속 1이 탈퇴

[0. 0. 0. 1. 0. 1. 0. 1. 1. 0. 1. 1. 0. 1. 0. 1. 0. 0. 1. 1. 1. 1. 1. 1.
 0. 1. 1. 1. 0. 0. 1. 0. 0. 0. 1. 1. 1. 1. 0. 0. 1. 1. 0. 1. 1. 0. 1. 1.
 1. 0. 1. 0. 0. 1. 0. 1. 0. 1. 1. 1. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 1. 1.
 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 0. 0. 0. 0. 1. 0. 1. 0. 0. 1. 0.
 1. 1. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0.
 0. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 1. 1. 0. 0. 1. 0. 1. 0. 0. 1. 0. 1. 0. 1. 1. 0. 0. 1. 1. 1. 1.
 1. 1. 0. 1. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 1. 1.
 0. 1. 1. 1. 1. 0. 0. 0. 1. 0. 1. 0. 1. 1. 1. 0. 1. 0. 1. 0. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 1. 0. 0. 1. 1. 0. 1.
 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 1. 0. 1. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 0. 1. 1. 0. 0. 0. 0. 0.
 0. 1. 0. 1. 0. 1. 1. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1.
 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 0. 0. 1. 1.

In [669]:
result_test = pd.DataFrame({"y_test":y_test,"y_pred":y_test_pred})
result_test

,y_test,y_pred
1091,0.0,0.0
1786,0.0,0.0
1439,0.0,0.0
745,1.0,1.0
820,1.0,0.0
...,...,...
1170,0.0,0.0
938,1.0,0.0
948,1.0,1.0
1652,0.0,0.0


### 8 : 예측 모델을 평가하고, 모델 튜닝

In [670]:
# 모델 튜닝

correct = len(result_test.loc[result_test['y_test']==result_test['y_pred']])
data_count = len(result_test)
score_test = correct / data_count
print(score_test)

0.8931116389548693


In [671]:
print(df_clf.score(X_test,y_test))
print(df_clf.score(X_train,y_train))

0.8931116389548693
0.9797979797979798


In [672]:
X = pd.concat([exit,conti],ignore_index=True)
y = X['is_deleted']
del X['is_deleted']


X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=0)

# 훈련 
model = DecisionTreeClassifier(random_state=0,max_depth=5)
model.fit(X_train,y_train)
print(model.score(X_test,y_test))
print(model.score(X_train,y_train))

0.9144486692015209
0.9252217997465145


### 9 : 모델에 기여하고 있는 변수 확인

In [673]:
importance = pd.DataFrame({"feature_names":X.columns,"coefficient":model.feature_importances_})
importance

#     count_1   routine_flg, preiod
# 1개월전 이용횟수, 정기 이용여부, 재적기간이 Y값에 영향을 많이 줌

,feature_names,coefficient
0,count_1,0.348565
1,routine_flg,0.137263
2,preiod,0.512198
3,campaign_name_입회비무료,0.000000
4,campaign_name_입회비반액할인,0.000000
5,class_name_종일,0.000000
6,class_name_주간,0.000000
7,gender_F,0.001974


### 10 : 회원의 탈퇴 예측

In [674]:
count_1 = 3
routing_flg =1
period = 10 
campaign_name = "입회비무료"
class_name = "종일"
gender = "M"

In [675]:
if campaign_name == "입회비반값할인":
    campaign_name_list = [1,0]
elif campaign_name == "입회비무료":
    campaign_name_list = [0,1]
elif campaign_name == "일반":
    campaign_name_list = [0,0]
    
if class_name == "종일":
    class_name_list = [1,0]
elif class_name == "주간":
    class_name_list = [0,1]
elif class_name == "야간":
    class_name_list = [0,0]
    
if gender == "F":
    gender_list = [1]
elif gender == "M":
    gender_list = [0]
    
    
input_data = [count_1,routing_flg,period]
input_data.extend(campaign_name_list) 
input_data.extend(class_name_list) 
input_data.extend(gender_list)
input_data    

[3, 1, 10, 0, 1, 1, 0, 0]

In [676]:
print(model.predict([input_data]))
print(model.predict_proba([input_data]))

[1.]
[[0. 1.]]
